In [1]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
%matplotlib inline

In [2]:
# Query modeling_data2 table from PostgreSQL database

try:
    conn = psycopg2.connect(user="cohort17",
                            password="Cohort17Movies",
                            host="moviesdb.ce8d6g1pa5lm.us-east-1.rds.amazonaws.com",
                            port="5432",database="moviesdb")

    dbquery = "select * from modeling_data2"
    
    movies = pd.read_sql_query(dbquery, conn)

except (Exception, psycopg2.Error) as error :
    print ("Error while fetching data from PostgreSQL", error)

finally:
    if(conn):
        conn.close()

In [3]:
# Create week_number feature from release date, create input and output datasets, and create train/test split

movies['releasedate']=pd.to_datetime(movies['releasedate'])
movies['week_number']=movies['releasedate'].dt.week

X = movies.drop(['primarytitle','releasedate','domesticgross'], axis=1)
y = movies['domesticgross']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

In [4]:
# Impute missing values in training and test datasets with median value from test dataset

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median', copy=False, fill_value=None)
median_imputer.fit(X_train)
X_train = median_imputer.transform(X_train)
X_test = median_imputer.transform(X_test)

In [5]:
# Scale values of training and test datasets using min/max scaler fit on training dataset

scaler = MinMaxScaler(copy=False)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.6145472247205245

In [7]:
en = ElasticNetCV(max_iter=1000, cv=5, random_state=0, l1_ratio=[.1, .5, .7, .9, .95, .99, 1])
en.fit(X_train, y_train)
en.score(X_test, y_test)

0.555498179328866

In [8]:
gbm = GradientBoostingRegressor(random_state=0)
gbm.fit(X_train,y_train)
gbm.score(X_test,y_test)

0.5916038127269018

In [9]:
#rf = RandomForestRegressor()
#random_grid = {'bootstrap': [True, False],
#               'max_depth': [5,10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#               'max_features': ['auto', 'sqrt'],
#               'min_samples_leaf': [1, 2, 4, 8, 10, 20],
#               'min_samples_split': [2, 5, 10, 20],
#               'n_estimators': [50, 100, 150, 200]}
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
#rf_random = RandomizedSearchCV(estimator = rf, 
#                               param_distributions = random_grid, 
#                               n_iter = 150, 
#                               cv = 5, 
#                               verbose=2, 
#                               random_state=0, 
#                               n_jobs = -1)
# Fit the random search model
#rf_random.fit(X_train, y_train)
#rf_random.score(X_test,y_test)

In [10]:
#rf_random.best_estimator_